# World Bank Ed Stats Model Building

This data has been collected and encoded by the World Bank as indicators for educational performance and attainment, as well as expenditure on education, since 1970. The data spans the countries of the world and aggregates some regions and socio-economic distinctions. The dataset is sparse, however, with a majority of null values. As I prepare the dataset for modeling, I have three objectives:

### Arrange the Data 
### Context and Visualizations
### Null Handling and Feature Selection

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

%matplotlib inline

#Open and read the CSV file to a Dataframe
data = pd.read_csv('EdStatsData.csv')

# Save a dictionary matching the indicator code to its indicator name
ind_codes = data['Indicator Code'].unique().tolist()
ind_names = data['Indicator Name'].unique().tolist()
indicator_dict = {k:v for (k,v) in zip(ind_codes, ind_names)}
#data.dropna(axis=1, thresh=75000, inplace=True)

## Arrange the Data
First, we will remove some of the unnecessary columns, like country codes, indicator names, and future years. There are far too many indicators to check the missing values for each, so we will isolate the indicator group from the prefix to the indicator code. Then we will separate the data frame into countries, regions, and socio-economic levels. Our immediate interest is the set of countries.

In [2]:
# Remove columns for years that have yet to happen
data.drop(['Country Code', 'Indicator Name', '2020', '2025', '2030', '2035', '2040', '2045', '2050', '2055',
          '2060', '2065', '2070', '2075', '2080', '2085', '2090', '2095', '2100', 'Unnamed: 69'], axis=1, inplace=True)
data.head()

,Country Name,Indicator Code,1970,1971,1972,1973,1974,1975,1976,1977,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Arab World,UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,SE.PRM.TENR,54.822121,54.894138,56.209438,57.267109,57.991138,59.36554,60.999962,61.92268,...,84.011871,84.195961,85.211998,85.24514,86.101669,85.51194,85.320152,NaN,NaN,NaN


In [3]:
# Create a column to separate out the group from the indicator
data['Indicator Group'] = [s.split('.')[0] for s in data['Indicator Code']] 

In [4]:
# Separate data frame by country, region, or socio-economic distinction
regions = ['Arab World', 'East Asia & Pacific', 'East Asia & Pacific (excluding high income)', 'Euro area', 'Europe & Central Asia', 
 'Europe & Central Asia (excluding high income)', 'European Union', 'Latin America & Caribbean', 'Latin America & Caribbean (excluding high income)', 
  'Middle East & North Africa', 'Middle East & North Africa (excluding high income)', 'Middle income', 'North America', 'South Asia', 
           'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)', 'OECD members', 'World']

income_levels = ['Heavily indebted poor countries (HIPC)', 'High income', 'Least developed countries: UN classification', 'Low & middle income', 
 'Low income', 'Lower middle income', 'Upper middle income']

reg_df = data[data['Country Name'].isin(regions)]

inc_df = data[data['Country Name'].isin(income_levels)]

cntry_df = data[~data['Country Name'].isin(regions)]
cntry_df = cntry_df[~cntry_df['Country Name'].isin(income_levels)]

# Remove initial data frame from working memory  
del data

cntry_df.head()

,Country Name,Indicator Code,1970,1971,1972,1973,1974,1975,1976,1977,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Indicator Group
91625,Afghanistan,UIS.NERA.2,NaN,NaN,NaN,NaN,7.05911,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,47.436790,50.627232,NaN,NaN,NaN,UIS
91626,Afghanistan,UIS.NERA.2.F,NaN,NaN,NaN,NaN,2.53138,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,34.073261,37.641541,NaN,NaN,NaN,UIS
91627,Afghanistan,UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,0.22154,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.567060,0.598370,NaN,NaN,NaN,UIS
91628,Afghanistan,UIS.NERA.2.M,NaN,NaN,NaN,NaN,11.42652,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,60.087059,62.906952,NaN,NaN,NaN,UIS
91629,Afghanistan,SE.PRM.TENR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SE


### Reindexing

Now that we have the data frame with just the countries of the world for years that have actually occurred, it is time to get the indicators set as the columns, grouped by their indicator group. 

In [5]:
# Set the desired columns as indices
cntry_df.set_index(['Country Name', 'Indicator Group', 'Indicator Code'], inplace=True)

In [6]:
# Transpose to get the row index to be the year
df_t = cntry_df.transpose()

In [7]:
# Stack the Country Name column as a secondary index for the rows
df_t = df_t.stack('Country Name')
df_t.head()

Indicator Group                     BAR                                       \
Indicator Code      BAR.NOED.1519.FE.ZS BAR.NOED.1519.ZS BAR.NOED.15UP.FE.ZS   
     Country Name                                                              
1970 Afghanistan                  91.44            77.08               97.21   
     Albania                      26.56            21.70               41.88   
     Algeria                      69.70            52.90               87.39   
     American Samoa                 NaN              NaN                 NaN   
     Andorra                        NaN              NaN                 NaN   

Indicator Group                                                            \
Indicator Code      BAR.NOED.15UP.ZS BAR.NOED.2024.FE.ZS BAR.NOED.2024.ZS   
     Country Name                                                           
1970 Afghanistan               88.81                94.8            78.40   
     Albania                   37.92                28.2            28.38   
     Algeria                   73.64                69.7            52.90   
     American Samoa              NaN                 NaN              NaN   
     Andorra                     NaN                 NaN              NaN   

Indicator Group                                                               \
Indicator Code      BAR.NOED.2529.FE.ZS BAR.NOED.2529.ZS BAR.NOED.25UP.FE.ZS   
     Country Name                                                              
1970 Afghanistan                  98.60            91.09               99.25   
     Albania                      31.77            28.91               48.28   
     Algeria                      91.50            77.30               95.90   
     American Samoa                 NaN              NaN                 NaN   
     Andorra                        NaN              NaN                 NaN   

Indicator Group                       ...                  UIS  \
Indicator Code      BAR.NOED.25UP.ZS  ... UIS.XUNIT.US.4.FSGOV   
     Country Name                     ...                        
1970 Afghanistan               94.22  ...                  NaN   
     Albania                   43.80  ...                  NaN   
     Algeria                   84.40  ...                  NaN   
     American Samoa              NaN  ...                  NaN   
     Andorra                     NaN  ...                  NaN   

Indicator Group                                                      \
Indicator Code      UIS.XUNIT.US.56.FSGOV UIS.XUNIT.USCONST.1.FSGOV   
     Country Name                                                     
1970 Afghanistan                      NaN                       NaN   
     Albania                          NaN                       NaN   
     Algeria                          NaN                       NaN   
     American Samoa                   NaN                       NaN   
     Andorra                          NaN                       NaN   

Indicator Group                                                           \
Indicator Code      UIS.XUNIT.USCONST.2.FSGOV UIS.XUNIT.USCONST.23.FSGOV   
     Country Name                                                          
1970 Afghanistan                          NaN                        NaN   
     Albania                              NaN                        NaN   
     Algeria                              NaN                        NaN   
     American Samoa                       NaN                        NaN   
     Andorra                              NaN                        NaN   

Indicator Group                                                          \
Indicator Code      UIS.XUNIT.USCONST.3.FSGOV UIS.XUNIT.USCONST.4.FSGOV   
     Country Name                                                         
1970 Afghanistan                          NaN                       NaN   
     Albania                              NaN                       NaN   
     Algeria       

In [8]:
# Unstack the year index and stack it back, nested within the country name
#df_t = df_t.unstack(0).stack()
#df_t.head()

In [9]:
# Create a list of indicator groups and iterate through that list to determine the percent of null values for that group
cols = df_t.columns
lst = [e[0] for e in cols]
col_lst = list(set(lst))

# Create a dictionary with the group as the key and the percent sparsity as the value in a list for the key
group_dict = {}

for group in col_lst:
    na_pct = np.sum(df_t[group].isna().sum()) / df_t[group].size
    group_dict[group] = []
    group_dict[group].append(na_pct)
    print('{} Null Percentage: {:2f}'.format(group, na_pct))

SP Null Percentage: 0.466671
LO Null Percentage: 0.994228
SL Null Percentage: 0.801463
UIS Null Percentage: 0.869046
PRJ Null Percentage: 0.967372
HH Null Percentage: 0.988921
SABER Null Percentage: 0.998478
NY Null Percentage: 0.375971
OECD Null Percentage: 0.961929
BAR Null Percentage: 0.872629
SE Null Percentage: 0.680125
IT Null Percentage: 0.645111
SH Null Percentage: 0.405946
XGDP Null Percentage: 0.882260


## Context 
That was a lot of missing data. I figured there had to be some reason for that many null values, so I did some investigation into the programs that collected the data. I will try to show a summary of my findings in a table to explain some of the sparsity and why I will be dropping some values.

In [10]:
# Create a list of column names for relevant information and add the information for each column to the dictionary. Make df.
column_names = ['Percent Missing', 'Data Type', 'Collected Since', 'Collected Through', 'Collection Interval', 
                'Participating Countries(count)', 'Country Distribution', 'Comments', 'Next Steps']

group_dict['PRJ'].extend(['Educational Attainment Projections', 2010, 2100, 5, '195', 'All',
                          'Not interested in projection data', 'Drop'])
group_dict['BAR'].extend(['Educational Attainment', 1970, 2010, 5, '146', 'Unclear', 
                          'Includes percentages, counts, and averages by gender and age group', 'Explore similarity to UIS.EA'])
group_dict['IT'].extend(['Access to IT', 1975, 2016, 1, 'Unclear', 'Unclear', 
                         'Percent computer owners and internet users', 'Drop'])
group_dict['NY'].extend(['Economic Policy & Debt', 1990, 2017, 1, '150+', 'Unclear', 
                          'Maybe interested depending on distribution of values', 'Explore sparsity'])
group_dict['SABER'].extend(['Various Educational Indicators', 2012, 2016, 1, '130', 'Dispersed', 
                          'Each participating country reports a small subset of indicators (1-4 of 10)', 'Drop'])
group_dict['LO'].extend(['Learning Outcomes', 1991, 2016, 3, 'Varies', 'Varies', 
                          'Subgroups vary widely on collection date, interval, countries, and outcomes', 'Explore LO.EGRA'])
group_dict['SL'].extend(['Social Protection & Labor', 1991, 2016, 1, 'Varies', 'Varies', 
                          'Unemployment and education attainment of workforce by gender', 'Drop'])
group_dict['UIS'].extend(['Various Educational Indicators', 2000, 2016, 1, 'Varies', 'Varies', 
                          'Includes enrollment, attainment, teacher training and more', 
                          'Explore sparsity & similarities to other indicators, combine with SE'])
group_dict['SP'].extend(['Population & Health', 1970, 2016, 1, 'Varies', 'Varies', 
                          'Not interested in population and health', 'Drop'])
group_dict['OECD'].extend(['Teacher Salaries', 2000, 2016, 1, '36', 'OECD members', 
                          'No meaningful comparison for non-OECD countries', 'Drop'])
group_dict['SH'].extend(['HIV and Child Mortality', 1970, 2016, 1, '190', 'All', 
                          'Not interested in mortality and HIV data', 'Drop'])
group_dict['SE'].extend(['Various Educational Indicators', 1970, 1999, 1, 'Varies', 'Varies', 
                          'Includes enrollment, attainment, teacher training and more', 
                          'Explore sparsity & similarities to other indicators, combine with UIS'])
group_dict['HH'].extend(['Education Equality', 1984, 2016, 1, '90+', 'Dispersed', 
                          'Attendance and completion rates from two programs', 'Explore Sparsity'])
group_dict['XGDP'].extend(['School Funding', 2000, 2016, 1, '190', 'All', 
                          'Covers secondary and tertiary education finance', 'Drop'])

context_table = pd.DataFrame.from_dict(group_dict, orient='index', columns=column_names)
context_table

,Percent Missing,Data Type,Collected Since,Collected Through,Collection Interval,Participating Countries(count),Country Distribution,Comments,Next Steps
SP,0.466671,Population & Health,1970,2016,1,Varies,Varies,Not interested in population and health,Drop
LO,0.994228,Learning Outcomes,1991,2016,3,Varies,Varies,"Subgroups vary widely on collection date, inte...",Explore LO.EGRA
SL,0.801463,Social Protection & Labor,1991,2016,1,Varies,Varies,Unemployment and education attainment of workf...,Drop
UIS,0.869046,Various Educational Indicators,2000,2016,1,Varies,Varies,"Includes enrollment, attainment, teacher train...",Explore sparsity & similarities to other indic...
PRJ,0.967372,Educational Attainment Projections,2010,2100,5,195,All,Not interested in projection data,Drop
HH,0.988921,Education Equality,1984,2016,1,90+,Dispersed,Attendance and completion rates from two programs,Explore Sparsity
SABER,0.998478,Various Educational Indicators,2012,2016,1,130,Dispersed,Each participating country reports a small sub...,Drop
NY,0.375971,Economic Policy & Debt,1990,2017,1,150+,Unclear,Maybe interested depending on distribution of ...,Explore sparsity
OECD,0.961929,Teacher Salaries,2000,2016,1,36,OECD members,No meaningful comparison for non-OECD countries,Drop
BAR,0.872629,Educational Attainment,1970,2010,5,146,Unclear,"Includes percentages, counts, and averages by ...",Explore similarity to UIS.EA


### Next Steps
The research I did is not exhaustive, but provides a good first look to help me reduce features in which I am uninterested. I want to work with data that has 10 or more years of history and could be descriptive of most countries (not just one or two regions or the set of OECD nations). This exploration also indicated where I could look to combine features or reduce them for redundancy. So now I will work on the next steps indicated in the last column of the above table.

In [11]:
# Drop columns not of interest
df_t.drop(['IT', 'SABER', 'PRJ', 'XGDP', 'SP', 'OECD', 'SL', 'SH'], axis=1, inplace=True)

In [12]:
# Drop Learning Outcomes that are note the literacy indicator
df_t.drop([e for e in df_t['LO'] if not e.startswith('LO.EGRA.')], axis=1, level=1, inplace=True)

In [13]:
# Create a data frame where null values are indicated by 0 all other values by 1
lo_sparsity = df_t['LO']
lo_sparsity = lo_sparsity.applymap(lambda x: 1 if -100<=x<=100 else 0)
lo_sparsity = lo_sparsity.T
lo_sparsity.head()

1970                                         \
Country Name          Afghanistan Albania Algeria American Samoa Andorra   
Indicator Code                                                             
LO.EGRA.CLPM.AFA.2GRD           0       0       0              0       0   
LO.EGRA.CLPM.AFA.3GRD           0       0       0              0       0   
LO.EGRA.CLPM.AMH.2GRD           0       0       0              0       0   
LO.EGRA.CLPM.AMH.3GRD           0       0       0              0       0   
LO.EGRA.CLPM.BMN.2GRD           0       0       0              0       0   

                                                                          ...  \
Country Name          Angola Antigua and Barbuda Argentina Armenia Aruba  ...   
Indicator Code                                                            ...   
LO.EGRA.CLPM.AFA.2GRD      0                   0         0       0     0  ...   
LO.EGRA.CLPM.AFA.3GRD      0                   0         0       0     0  ...   
LO.EGRA.CLPM.AMH.2GRD      0                   0         0       0     0  ...   
LO.EGRA.CLPM.AMH.3GRD      0                   0         0       0     0  ...   
LO.EGRA.CLPM.BMN.2GRD      0                   0         0       0     0  ...   

                             2016                        2017                  \
Country Name          Yemen, Rep. Zambia Zimbabwe Afghanistan Bangladesh Fiji   
Indicator Code                                                                  
LO.EGRA.CLPM.AFA.2GRD           0      0        0           0          0    0   
LO.EGRA.CLPM.AFA.3GRD           0      0        0           0          0    0   
LO.EGRA.CLPM.AMH.2GRD           0      0        0           0          0    0   
LO.EGRA.CLPM.AMH.3GRD           0      0        0           0          0    0   
LO.EGRA.CLPM.BMN.2GRD           0      0        0           0          0    0   

                                                               
Country Name          Liberia Sierra Leone Tajikistan Ukraine  
Indicator Code                                                 
LO.EGRA.CLPM.AFA.2GRD       0            0          0       0  
LO.EGRA.CLPM.AFA.3GRD       0            0          0       0  
LO.EGRA.CLPM.AMH.2GRD       0            0          0       0  
LO.EGRA.CLPM.AMH.3GRD       0            0          0       0  
LO.EGRA.CLPM.BMN.2GRD       0            0          0       0  

[5 rows x 10175 columns]

In [14]:
# Remove columns where there are no values and put the year in the index
lo_sparsity = lo_sparsity[['2008', '2009', '2010', '2011', '2012', '2013', '2014']]
lo_sparsity = lo_sparsity.stack(0)
lo_sparsity = lo_sparsity.unstack(0).stack()

In [15]:
# Drop countries with no values and create a dictionary to show which countries had how many values during this period
sp_dict = {}
for column in lo_sparsity.columns:
    if lo_sparsity[column].sum() == 0:
        lo_sparsity.drop(column, axis=1, inplace=True)
    else:
        sp_dict[column] = lo_sparsity[column].sum()
sp_dict

{'Egypt, Arab Rep.': 14,
 'Ethiopia': 96,
 'Ghana': 84,
 'Guyana': 30,
 'Indonesia': 7,
 'Jordan': 28,
 'Malawi': 54,
 'Mali': 36,
 'Nicaragua': 26,
 'Philippines': 33,
 'Rwanda': 29,
 'Senegal': 6,
 'Tanzania': 14,
 'West Bank and Gaza': 8,
 'Zambia': 51}

For over 400 indicators through the course of 7 years, I have values for 15 countries ranging from 6 values to 96 values. These indicators do not meet the criteria I was hoping for in this analysis. Therefore I will be dropping them from the data frame.

In [16]:
# Drop all LO columns as they are no longer of interest
df_t.drop('LO', axis=1, inplace=True)

In [17]:
# Drop the Africa Dataset, since we are interested in data for global comparison
df_t.drop([e for e in df_t['UIS'] if e.startswith('UIS.AFR')], axis=1, level=1, inplace=True)

In [18]:
# Create dictionaries of indicators to names and names to indicators for groups that may have common indicators
usb_dict = {k:v for k, v in indicator_dict.items() if k.startswith('UIS') or k.startswith('BAR') or k.startswith('SE')}
usb_dict = {k:v for k, v in usb_dict.items() if not k.startswith('UIS.AFR')}
reversed_usb = {v:k for k,v in usb_dict.items()}

In [19]:
# Create a dictionary of indicator codes of potentially similar indicators based on close matches of indicator names
import difflib
u = [v for k, v in usb_dict.items() if k.startswith('UIS')]
s = [v for k, v in usb_dict.items() if k.startswith('SE')]
b = [v for k, v in usb_dict.items() if k.startswith('BAR')]
usb_list = [u, s, b]
match_dict = {}
for i in range(len(usb_list) - 1):
    for name in usb_list[i]:
        matches = difflib.get_close_matches(name, usb_list[i+1], cutoff=0.97)
        if len(usb_list) - i > 2:
            matches.extend(difflib.get_close_matches(name, usb_list[i+2], cutoff=0.97))
        for match in matches:
            if reversed_usb[match] not in match_dict.keys():
                match_dict[reversed_usb[match]] = reversed_usb[name]
len(match_dict)

17

In [20]:
# Create a data frame to compare the indicators that are potential matches by their names, the total countries polled over
# which years, and the mean value of the measurements taken by each

def matches_to_df(df, matches, indicator_dict):
    '''A function designed to take a data frame, a dictionary of close matches for column names, and a dictionary matching
    codes to long form names in order to output a data frame that compares the values of the matching columns.'''
    df_list = []
    
    for k, v in matches.items():
        code_a, name_a, code_b, name_b = k, indicator_dict[k], v, indicator_dict[v]
        
        index_1 = df.index[df['SE', k] > 0].tolist()
        index_1 = [i[0] for i in index_1]
        index_1 = Counter(index_1)
        
        fyear_a, lyear_a = min(index_1.keys()), max(index_1.keys())
        cntry_a = np.mean(list(index_1.values()))
        mean_a = np.nanmean(df['SE', k])
        
        index_2 = df.index[df['UIS', v] > 0].tolist()
        index_2 = [i[0] for i in index_2]
        index_2 = Counter(index_2)
        
        fyear_b, lyear_b = min(index_2.keys()), max(index_2.keys())
        cntry_b = np.mean(list(index_2.values()))
        mean_b = np.nanmean(df['UIS', v])
        
        df_list.append([code_a, name_a, fyear_a, lyear_a, cntry_a, mean_a, code_b, name_b, fyear_b, lyear_b, cntry_b, mean_b])
        
    new_df = pd.DataFrame(df_list, columns=['Code1', 'Name1', 'Start_Year1', 'End_Year1', 'Countries1', 'Mean1', 
                                   'Code2', 'Name2', 'Start_Year2', 'End_Year2', 'Countries2', 'Mean2'])
    return new_df

matches_df = matches_to_df(df_t, match_dict, indicator_dict)
matches_df

,Code1,Name1,Start_Year1,End_Year1,Countries1,Mean1,Code2,Name2,Start_Year2,End_Year2,Countries2,Mean2
0,SE.XPD.PRIM.ZS,Expenditure on primary as % of government expe...,1970,2016,59.446809,36.742168,UIS.XPUBP.0,Expenditure on pre-primary as % of government ...,1998,2015,64.944444,6.630427
1,SE.SEC.ENRL.FE.VO.ZS,Percentage of female students in secondary edu...,1970,2016,98.531915,13.053949,UIS.GTVP.2.V.F,Percentage of female students in lower seconda...,1998,2014,45.823529,5.796023
2,SE.PRM.TCAQ.FE.ZS,Percentage of female teachers in primary educa...,1998,2016,52.789474,82.824042,UIS.TRTP.0.F,Percentage of female teachers in pre-primary e...,1998,2016,37.000000,70.936191
3,SE.TER.GRAD.AG.FE.ZS,Percentage of female graduates from tertiary e...,1998,2016,45.105263,2.351788,UIS.FOSGP.56.F600.M,Percentage of male graduates from tertiary edu...,1998,2015,51.388889,3.100323
4,SE.TER.GRAD.ED.FE.ZS,Percentage of female graduates from tertiary e...,1998,2015,56.333333,18.997019,UIS.FOSGP.56.F140.M,Percentage of male graduates from tertiary edu...,1998,2015,55.944444,9.648337
5,SE.TER.GRAD.EN.FE.ZS,Percentage of female graduates from tertiary e...,1998,2016,46.157895,4.497202,UIS.FOSGP.56.F500.M,Percentage of male graduates from tertiary edu...,1998,2015,56.277778,21.579777
6,SE.TER.GRAD.HL.FE.ZS,Percentage of female graduates from tertiary e...,1998,2016,47.105263,6.189694,UIS.FOSGP.56.F700.M,Percentage of male graduates from tertiary edu...,1998,2015,55.666667,6.767917
7,SE.TER.GRAD.HU.FE.ZS,Percentage of female graduates from tertiary e...,1998,2016,46.894737,11.431689,UIS.FOSGP.56.F200.M,Percentage of male graduates from tertiary edu...,1998,2015,55.500000,8.464959
8,SE.TER.GRAD.OT.FE.ZS,Percentage of female graduates from tertiary e...,1998,2016,20.736842,0.964218,UIS.FOSGP.56.FUK.M,Percentage of male graduates from tertiary edu...,1998,2015,24.944444,2.290472
9,SE.TER.GRAD.SC.FE.ZS,Percentage of female graduates from tertiary e...,1998,2016,48.789474,28.264670,UIS.FOSGP.56.F400.M,Percentage of male graduates from tertiary edu...,1998,2015,55.500000,11.433544


### Knowing when to change approaches

After much trial and error surrounding the above approach to algorithmically get similar indicators for combining, I have come to the realization that whatever cutoff pecentage I set, I will be left with a mix of good matches and bad matches. I cannot rely on python to decide which indicators could be combined. I will go on to see which indicators contain the most dense data and then determine whether those indicators have an analog in another indicator group that can be used to impute null values.

In [ ]:
'''# predicting sparsity in the most recent three to five years
# research the programs
# heatmap with year and indicators and region white for value and black for missingness
# timeseries heatmap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA'''

In [ ]:
'''sklearn_pca = PCA(n_components=10)
Y_sklearn = sklearn_pca.fit_transform(df_t)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_)'''